In [1]:
# !pip install datasets

In [2]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
# from datasets import load_dataset
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import pandas as pd
import random
from torch.utils.data import DataLoader
import numpy as np
import os

# Set seed for reproducibility
def set_seed_(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed_(42)
set_seed(42)

[1760641109.723753] [bfd74565809f:107555:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [4]:
!nvidia-smi

Thu Oct 16 18:58:36 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   47C    P0              65W / 300W |   1978MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/workspace/Assignment2/final_llm


In [6]:
test_dataset = pd.read_csv('../dataset/test_translit.txt', delimiter='\t', encoding='utf-8', na_filter=False, header=None)
print(test_dataset.shape)
ENG_TEST = test_dataset.iloc[:, 0].to_numpy()
HIN_TEST = test_dataset.iloc[:, 1].to_numpy()

(10112, 2)


In [7]:
print(ENG_TEST[0], HIN_TEST[0])

maitrologist मैट्रोलॉजिस्ट


In [19]:
from tqdm import tqdm
import torch

def get_pos_responses_with_one_shot_gemma(words, model, tokenizer, device='cuda', max_new_tokens=128):
    prompt_template = """
<bos><start_of_turn>user

You are a Transliterator.  
Your task is to transliterate each English word into Hindi (Devanagari script).  
Abbreviations like "USA" should be transliterated letter by letter (e.g., यूएसए).

### Instructions:
- Input: a word in English.  
- Output: only the Hindi transliteration. Do NOT add explanations, extra text, or punctuation.

### Examples:
Input: janamdin
Output: जन्मदिन

Input: ram
Output: राम

Input: pushpak
Output: पुष्पक

Input: shalaka
Output: शलाका

Input: nri
Output: एनआरआई

Input: doctor
Output: डॉक्टर

### Now transliterate the following word:
Only provide the transliteration, no extra text or explanation.
Input: {word}
Output:
<end_of_turn> <start_of_turn>model
"""
    responses = []
    model.to(device)
    model.eval()

    for word in tqdm(words):
        prompt = prompt_template.format(word=word)
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )
        text = tokenizer.decode(output[0], skip_special_tokens=True)
        # Keep only model part (after "Output:" or last "model")
        clean_text = text.split("Output:")[-1].split("<end_of_turn>")[0].strip()
        responses.append(clean_text)

    return responses

In [20]:
words = ["shalaka", "kaustubh", "ram", "pushpak", "usa", "paradise", "CSE"]

In [21]:
responses = get_pos_responses_with_one_shot(words, max_new_tokens=128)
print(responses)

  0%|                                                                                             | 0/7 [00:00<?, ?it/s]


Unsupported: call_method GetAttrVariable(UserDefinedObjectVariable(AttentionInterface), _global_mapping) __getitem__ (ConstantVariable(),) {}

from user code:
   File "/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py", line 959, in wrapper
    output = func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/gemma2/modeling_gemma2.py", line 548, in forward
    outputs: BaseModelOutputWithPast = self.model(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py", line 1083, in wrapper
    outputs = func(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/gemma2/modeling_gemma2.py", line 452, in forward
    layer_outputs = decoder_layer(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1541, in 

In [32]:
import ast

# Convert string to list of tuples
word_tag_list = ast.literal_eval(responses[0])

# Find max width for formatting
word_width = max(len(word) for word, _ in word_tag_list)
tag_width = max(len(tag) for _, tag in word_tag_list)

# Print header
print(f"+{'-'*(word_width+2)}+{'-'*(tag_width+2)}+")
print(f"| {'Word':<{word_width}} | {'POS Tag':<{tag_width}} |")
print(f"+{'-'*(word_width+2)}+{'-'*(tag_width+2)}+")

# Print rows
for word, tag in word_tag_list:
    print(f"| {word:<{word_width}} | {tag:<{tag_width}} |")

# Print footer
print(f"+{'-'*(word_width+2)}+{'-'*(tag_width+2)}+")

+-------+------+
| Word  | POS Tag |
+-------+------+
| Time  | NOUN |
| flies | VERB |
| like  | ADP  |
| an    | DET  |
| arrow | NOUN |
| .     | .    |
+-------+------+


In [51]:
words = ENG_TEST

In [53]:
import pandas as pd

# Assume these are defined
# ENG_TEST = [...]  # English words
# HIN_TEST = [...]  # Hindi words

# Get responses from your one-shot transliteration function
responses = get_pos_responses_with_one_shot(words, max_new_tokens=128)

100%|█████████████████████████████████████████████████████████████████████████████| 10112/10112 [58:38<00:00,  2.87it/s]


In [54]:
# Create DataFrame
df = pd.DataFrame({
    'eng': ENG_TEST,
    'hin': HIN_TEST,
    'original_response_mistral': responses
})

# Save as CSV
df.to_csv("gemma_responses.csv", index=False, encoding='utf-8')

# # Optional: save as Excel
# df.to_excel("gemma_responses.xlsx", index=False, encoding='utf-8')

print("Saved responses to mistral_responses.csv and .xlsx")

TypeError: NDFrame.to_excel() got an unexpected keyword argument 'encoding'

In [ ]:
# Create DataFrame
df = pd.DataFrame({
    'original_response_gemma': list(responses)
})

# Save as CSV
df.to_csv("gemma_responses1.csv", index=False, encoding='utf-8')

# Optional: save as Excel
df.to_excel("gemma_responses1.xlsx", index=False, encoding='utf-8')

print("Saved responses to gemma_responses1.csv and .xlsx")

In [ ]:
import os
print(os.getcwd())

In [ ]:
# !pip install sentencepiece

In [11]:
# Load the first causal LM for pos tagging
pos_model_name = 'google/gemma-2-9b-it'

pos_model = AutoModelForCausalLM.from_pretrained(pos_model_name)
pos_tokenizer = AutoTokenizer.from_pretrained(pos_model_name)
pos_tokenizer.padding_side='left'

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.20s/it]


In [12]:
# Ensure pad_token_id is set to eos_token_id if not already set
if pos_tokenizer.pad_token_id is None:
    pos_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    pos_model.resize_token_embeddings(len(pos_tokenizer))


device_stereo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pos_model.to(device_stereo)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): G

In [13]:
print(device_stereo)

cuda
